# Niraj GPT


# feeding the model with some baby conversation to learn from

In [16]:
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
import requests

In [17]:
with open("dialogs.txt", "r", encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

In [18]:
data = []
for i in range(0, len(lines) - 1, 2):
    input_text = lines[i].strip().lower()
    output_text = lines[i+1].strip().lower()
    data.append((input_text, output_text))

In [19]:
# Build vocabulary
all_text = [text for pair in data for text in pair]
words = set(" ".join(all_text).split())
word2idx = {word: i+1 for i, word in enumerate(words)}  # Start from 1
word2idx["<PAD>"] = 0  # Add padding token
idx2word = {i: word for word, i in word2idx.items()}


### Encode Sentences


In [20]:
def encode(sentence):
    return [word2idx[word] for word in sentence.split()]

X = [encode(pair[0]) for pair in data]
Y = [encode(pair[1]) for pair in data]


X = [torch.tensor(x) for x in X]
Y = [torch.tensor(y) for y in Y]

X_pad = pad_sequence(X, batch_first=True, padding_value=0)
Y_pad = pad_sequence(Y, batch_first=True, padding_value=0)


In [21]:

class MiniNirajGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out
    
vocab_size = len(word2idx)
model = MiniNirajGPT(vocab_size=vocab_size, embed_dim=16, hidden_dim=32)


In [23]:


# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.000001)

for epoch in range(1000):
    total_loss = 0

    for x, y in zip(X_pad, Y_pad):
        if x.size(0) != y.size(0):  # 🛡️ Skip mismatched pairs
            continue

        x = x.unsqueeze(0)  # Shape: [1, seq_len]
        y = y.unsqueeze(0)

        out = model(x)

        out = out.view(-1, vocab_size)
        y = y.view(-1)

        loss = loss_fn(out, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    if epoch % 50 == 49:
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 50, Loss: 0.0000
Epoch 100, Loss: 0.0000
Epoch 150, Loss: 0.0000
Epoch 200, Loss: 0.0000
Epoch 250, Loss: 0.0000
Epoch 300, Loss: 0.0000
Epoch 350, Loss: 0.0000
Epoch 400, Loss: 0.0000
Epoch 450, Loss: 0.0000
Epoch 500, Loss: 0.0000
Epoch 550, Loss: 0.0000
Epoch 600, Loss: 0.0000
Epoch 650, Loss: 0.0000
Epoch 700, Loss: 0.0000
Epoch 750, Loss: 0.0000
Epoch 800, Loss: 0.0000
Epoch 850, Loss: 0.0000
Epoch 900, Loss: 0.0000
Epoch 950, Loss: 0.0000
Epoch 1000, Loss: 0.0000


In [24]:
def chat(input_text):
    model.eval()
    with torch.no_grad():
        words = input_text.lower().split()

        # Handle unknown words using <PAD> token
        input_ids = [word2idx[word] if word in word2idx else word2idx["<PAD>"] for word in words]

        input_tensor = torch.tensor(input_ids).unsqueeze(0)

        output = model(input_tensor)

        # Grab most likely word at each position
        output_ids = output.argmax(dim=2)  # Shape: [1, seq_len]

        # ⚠️ .squeeze() can turn it into an int if it's only 1 word, so we use .tolist() carefully:
        if output_ids.numel() == 1:
            output_ids = [output_ids.item()]  # make it a list with 1 number
        else:
            output_ids = output_ids.squeeze().tolist()

        response_words = [idx2word.get(idx, "<UNK>") for idx in output_ids]
        response = " ".join(response_words)

        return response


In [25]:
print("MiniGeorgeGPT is ready! Type 'quit' to stop.\n")
while True:
    msg = input("You: ")
    if msg.lower() == "quit":
        break
    reply = chat(msg)
    print("MiniGeorgeGPT:", reply)


MiniGeorgeGPT is ready! Type 'quit' to stop.

MiniGeorgeGPT: cleaner. height some. jokes. careful.
MiniGeorgeGPT: noise?
